I trained the model using a public dataset ,the dataset is part of Sentiment140 student data found at [link](https://docs.google.com/file/d/0B04GJPshIjmPRnZManQwWEdTZjg/edit?resourcekey=0-betyQkEmWZgp8z0DFxWsHw)
I edited the training csv file to create new columns names was not part  of the original and that is what i used to train this model the csv is found at [My drive](https://drive.google.com/file/d/1N4S0xiuP1GnpZcoJiUADQPfg2QE0kigZ/view?usp=sharing)

In [ ]:
# Import the necessary libraries
import tensorflow
import keras

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import nltk



nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Load the preprocessed dataset
encoding = 'ISO-8859-1'
train_df=pd.read_csv('/content/drive/MyDrive/train.csv', encoding=encoding)

In [ ]:
train_df

,polarity,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew
...,...,...
1599994,1,Just woke up. Having no school is the best fee...
1599995,1,TheWDB.com - Very cool to hear old Walt interv...
1599996,1,Are you ready for your MoJo Makeover? Ask me f...
1599997,1,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
# Tokenizing
max_features = 1000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(train_df['text'].values)
train_tokens = tokenizer.texts_to_sequences(train_df['text'].values)
# Pad sequences to make them equal length
train_tokens = pad_sequences(train_tokens)

In [ ]:
# convert the sentiment labels to categorical
train_labels = to_categorical(train_df['polarity'] )

In [ ]:
train_labels.shape

(1599999, 2)

In [ ]:
#build the model

embed_dim = 196
lstm_out = 256

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = train_tokens.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.1, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))



In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 196)           196000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 196)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 256)               463872    
                                                                 
 dense (Dense)               (None, 2)                 514       
                                                                 
Total params: 660,386
Trainable params: 660,386
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#split the data for training and testing
X_train, X_test, y_train, y_test = train_test_split(train_tokens, train_labels, test_size=0.2, random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(1279999, 50) (1279999, 2)
(320000, 50) (320000, 2)


In [ ]:
# Training the model
batch_size = 1024
model.fit(X_train, y_train, epochs = 7, batch_size=batch_size, verbose = 1)

Epoch 1/7
1250/1250 [==============================] - 330s 258ms/step - loss: 0.4743 - accuracy: 0.7709
Epoch 2/7
1250/1250 [==============================] - 318s 255ms/step - loss: 0.4486 - accuracy: 0.7869
Epoch 3/7
1250/1250 [==============================] - 316s 252ms/step - loss: 0.4399 - accuracy: 0.7922
Epoch 4/7
1250/1250 [==============================] - 309s 248ms/step - loss: 0.4324 - accuracy: 0.7965
Epoch 5/7
1250/1250 [==============================] - 309s 248ms/step - loss: 0.4263 - accuracy: 0.8002
Epoch 6/7
1250/1250 [==============================] - 309s 248ms/step - loss: 0.4206 - accuracy: 0.8031
Epoch 7/7
1250/1250 [==============================] - 309s 247ms/step - loss: 0.4156 - accuracy: 0.8060


In [ ]:
# evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# print the evaluation results
print('Test loss:', loss)
print('Test accuracy:', accuracy * 100,'%')


10000/10000 [==============================] - 169s 17ms/step - loss: 0.4217 - accuracy: 0.8024
Test loss: 0.42174726724624634
Test accuracy: 80.2359402179718 %


In [ ]:
model.save('/content/drive/MyDrive/model.h5')